

### Generación de Modelos Predictivos

Este notebook se encarga de generar modelos predictivos.  Se puede correr en Google Colab o en el iMac.

Para usarlo en Google Colab, usar tipo_a_usar='GPU'
Para usarlo en el imac, usar tipo_a_usar='CPU'



In [2]:
tipo_a_usar='CPU'

In [3]:
import pandas as pd

In [4]:
if tipo_a_usar =='GPU':
  from google.colab import drive
  drive.mount('/content/drive')

Los datos de origen es el mismo sea desde Google Colab o desde el iMac.  La diferencia es que el primero usar referencia a Google Drive y el otro acceso directo desde el sistema operativo

In [5]:
if tipo_a_usar =='GPU':
  dfbase = pd.read_pickle('/content/drive/MyDrive//Innovaciones Tecnológicas Aplicadas/Universidad Autónoma de Occidente/GoogleColab/checkPoints/DATOS_LIMPIOS.pkl')

if tipo_a_usar == 'CPU':
  dfbase = pd.read_pickle('checkpoints/DATOS_LIMPIOS.pkl')


In [6]:
df = dfbase.copy()
df = df[df['LABEL']!=-1]

X = df.drop('LABEL', axis=1)  # Reemplaza 'target_column' con el nombre de tu columna objetivo
y = df['LABEL']  # Reemplaza 'target_column' con el nombre de tu columna objetivo

In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score

# Dividir los datos en conjuntos de entrenamiento (train) y prueba (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# BALANCAMOS X_train y y_train
#!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='auto')
X_train, y_train = smote.fit_resample(X_train, y_train)

# Definir los hiperparámetros a ajustar y sus posibles valores
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
}

# Crear el modelo de Random Forest
modelo_rf = RandomForestClassifier(random_state=42)

# Definir las métricas que deseas evaluar durante la búsqueda en cuadrícula
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Realizar la búsqueda en cuadrícula con validación cruzada en el conjunto de entrenamiento
grid_search = GridSearchCV(estimator=modelo_rf, param_grid=param_grid, cv=5, scoring=scoring, n_jobs=-1, refit='roc_auc')
grid_search.fit(X_train, y_train)

# Obtener el DataFrame de resultados de la búsqueda en cuadrícula
resultados_df = pd.DataFrame(grid_search.cv_results_)

# Encontrar la fila con el mejor resultado de ROC-AUC
mejor_roc_auc_idx = resultados_df['mean_test_roc_auc'].idxmax()

# Extraer los hiperparámetros correspondientes al mejor resultado
mejores_hiperparametros = resultados_df.loc[mejor_roc_auc_idx, ['param_n_estimators', 'param_max_depth',
                                                              'param_min_samples_split', 'param_min_samples_leaf',
                                                              'param_max_features']]

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros encontrados:")
print(mejores_hiperparametros.to_dict())

# Obtener el modelo con los mejores hiperparámetros
mejor_modelo_rf = grid_search.best_estimator_

# Evaluar el modelo en el conjunto de prueba
y_pred = mejor_modelo_rf.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred)
roc_auc_test = roc_auc_score(y_test, y_pred)

# Mostrar los resultados en el conjunto de prueba
print("Precisión (Accuracy) en prueba:", accuracy_test)
print("ROC-AUC en prueba:", roc_auc_test)


/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
259 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line

Mejores hiperparámetros encontrados:
{'param_n_estimators': 200, 'param_max_depth': None, 'param_min_samples_split': 2, 'param_min_samples_leaf': 1, 'param_max_features': 'log2'}
Precisión (Accuracy) en prueba: 0.9327286470143613
ROC-AUC en prueba: 0.9099763441488274


In [8]:
mejores_hiperparametros.head()

param_n_estimators          200
param_max_depth            None
param_min_samples_split       2
param_min_samples_leaf        1
param_max_features         log2
Name: 56, dtype: object

In [9]:
resultados_df.to_excel('RF Resultados balanceado.xlsx')